In [1]:
# !pip install -r reqirements.txt

In [2]:
import time
import warnings

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from lightgbm import LGBMRegressor
from pandas.tseries.offsets import MonthBegin
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

Esta función es para crear un dataset de prueba, con el número de países, productos y meses que le indiques.

In [3]:
def prepare_data(n_countries: int, n_products: int, n_months: int) -> pd.DataFrame:
    countries = [f"country_{i}" for i in range(n_countries)]
    products = [f"product_{i}" for i in range(n_products)]
    country_list = np.repeat(countries, n_products * n_months)
    product_list = np.tile(products, n_countries * n_months)
    dates = pd.date_range(start="2018-01-01", periods=n_months, freq="MS")
    sales = np.random.randint(100, 1001, size=len(country_list))

    # Create the DataFrame
    input_data = (
        pd.DataFrame()
        .assign(
            country=country_list,
            product=product_list,
            date=pd.to_datetime(np.tile(dates, n_countries * n_products)),
            sales=sales,
        )
        .sort_values(["country", "product", "date"])
    )
    print(f"Total number of data points: {len(input_data):,}")
    return input_data

Definimos las funciones de entrenamiento y predicción (secuencial y en paralelo).

In [4]:
def train_predict(models: list, data: pd.DataFrame) -> pd.DataFrame:
    warnings.filterwarnings("ignore")  # Ignoramos los warnings de skforecast
    # Predecimos 6 meses en el futuro
    prediction_horizon = 6

    # Creamos un diccionario para guardar las predicciones.
    predictions_dict = {
        "country": [],
        "product": [],
        "model": [],
        "date": [],
        "prediction": [],
    }

    # Agrupamos los datos por país y producto para luego iterar sobre los grupos.
    grouped = data.groupby(["country", "product"])

    # Iteramos sobre los modelos y los grupos de datos, entrenamos cada modelo para cada grupo.
    # Luego hacemos las predicciones y las guardamos en el diccionario.
    for model in models:
        for (country, product), group in grouped:
            forecaster = ForecasterAutoreg(regressor=model, lags=4)
            forecaster.fit(y=group["sales"])
            predictions = forecaster.predict(steps=prediction_horizon)

            # Generamos las fechas de las predicciones
            prediction_dates = [group["date"].max() + MonthBegin(i) for i in range(1, prediction_horizon + 1)]

            for date, predicted_sales in zip(prediction_dates, predictions):
                predictions_dict["country"].append(country)
                predictions_dict["product"].append(product)
                predictions_dict["model"].append(type(model).__name__)
                predictions_dict["date"].append(date)
                predictions_dict["prediction"].append(predicted_sales)

    # Convertimos el diccionario en un DataFrame
    predictions_dataframe = pd.DataFrame(predictions_dict)

    return predictions_dataframe

In [5]:
def sequential_train(input_data: pd.DataFrame, models: list) -> float:
    start_time = time.perf_counter()
    results_df = train_predict(models, input_data)
    end_time = time.perf_counter()
    # Para el benchmark solo nos interesa el tiempo de ejecución
    time_elapsed = end_time - start_time
    return time_elapsed

In [6]:
def parallel_train_auto(input_data: pd.DataFrame, models: list, n_cores: int) -> float:
    # Dividimos el DataFrame en grupos por país, en base al número de cores,
    # mapeamos cada país a un grupo para que todos los productos de un mismo país
    # siempre estén en el mismo grupo.
    country_list = input_data["country"].unique()
    country_to_group = {country: i % n_cores for i, country in enumerate(country_list)}
    input_data["group"] = input_data["country"].map(country_to_group)
    dataframes = [group.drop(columns="group") for _, group in input_data.groupby("group")]

    # Comprobamos que el número de grupos sea igual al número de cores.
    assert len(dataframes) == n_cores, "The number of dataframes must match the number of cores."

    start_time = time.perf_counter()
    # Luego usamos Parallel de joblib para entrenar los modelos en paralelo.
    predictions = Parallel(n_jobs=n_cores)(delayed(train_predict)(models, data) for data in dataframes)

    # Finalmente concatenamos los resultados en un solo DataFrame.
    predictions_dataframe = pd.concat([result for result in predictions])
    end_time = time.perf_counter()

    # Para el benchmark solo nos interesa el tiempo de ejecución
    time_elapsed = end_time - start_time
    return time_elapsed

Definimos la lista de modelos, se pueden añadir o quitar si quieres probar distintas combinaciones. Para los modelos que acepten `n_jobs` es mejor fijarlo en 1 para evitar problemas cuando se usan muchos cores.

In [7]:
MODELS = [
    LinearRegression(),
    GradientBoostingRegressor(random_state=42),
    XGBRegressor(random_state=42, n_jobs=1),
    LGBMRegressor(random_state=42, verbosity=-1, n_jobs=1),
]

Ejecutamos la comparativa. Recomiendo nunca usar todos los cores disponibles, puede dar problemas en la ejecución, mejor dejar siempre uno o dos cores de margen.

Importante: el número de países tiene que ser igual o superior al número de cores. 

In [8]:
input_data = prepare_data(n_countries=50, n_products=20, n_months=60)

print(f"Models: {', '.join([type(model).__name__ for model in MODELS])}")
print("--- Mean Execution Time ---")

time_sequential = sequential_train(models=MODELS, input_data=input_data)
print(f"Sequential: {time_sequential:0.2f} seconds")

# Ejecutamos el benchmark para el número de cores que queramos probar.
for n_cores in [2, 4, 6, 8]:
    time_parallel = parallel_train_auto(models=MODELS, input_data=input_data, n_cores=n_cores)
    print(f"Parallel ({n_cores} cores): {time_parallel:0.2f} seconds")

Total number of data points: 60,000
Models: LinearRegression, GradientBoostingRegressor, XGBRegressor, LGBMRegressor
--- Mean Execution Time ---
Sequential: 55.47 seconds
Parallel (2 cores): 28.18 seconds
Parallel (4 cores): 15.38 seconds
Parallel (6 cores): 11.58 seconds
Parallel (8 cores): 10.48 seconds
